# TRABALHO 1 - Computação Gráfica

><pre>
><b>Alunos:</b>                         <b>NUSP:</b>  
>Michelle Wingter da Silva       10783243
>Juliano Fantozzi                00000000</pre>

### Objetivo do trabalho:
Desenvolver um programa envolvendo os conceitos sobre transformações geométricas 2D,
conforme requisitos abaixo.
### Requisitos do trabalho:
1. O programa deve conter 5 ou mais objetos.
2. Cada objeto deve ter sua própria matriz de transformação composta pelas
transformações primárias.
3. As transformações geométricas de escala, rotação e translação devem ser
aplicadas.
4. Usar teclado para aplicar translação em pelo menos 1 objeto.
5. Usar teclado para aplicar rotação em pelo menos 1 objeto.
6. Usar teclado para aplicar escala em pelo menos 1 objeto.
7. Os seus objetos devem ser uma composição de diferentes primitivas. Na prática,
isso significa que o seu objeto não deve ser apenas um triângulo, quadrado e
círculo. Por exemplo, um cata-vento 2D com rotação nas pás será aceito (pois é um
objeto composto pelas primitivas). Apenas um triângulo rotacionando não será
aceito.
8. O programa deve ter um objetivo bem definido, ou seja, uma ação a ser executada
pelo usuário, com início, meio e fim.

# IMPLEMENTAÇÃO:

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import math

### Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(720, 600, "Cores", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Definindo uma variável chamada mat_transformation do tipo mat4 (matriz 4x4).
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base em uma matriz de transformação, conforme estudado na Aula05.

In [3]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,10.0);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [4]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [5]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [6]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [7]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [8]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [9]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [10]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


In [11]:
# preparando espaço para 3 vértices usando 2 coordenadas (x,y)
vertices = np.zeros(40, [("position", np.float32, 2)])

In [12]:
# preenchendo as coordenadas de cada vértice de uma ESTRELA 
'''
                            (-0.5, -0.5), # vertice 3 QUADRADO
                            (-0.5, +0.5), # vertice 4
                            (+0.0, -0.5), # vertice 5
                            (+0.5, +0.5), # vertice 6
                            (+0.5, -0.5), # vertice 7'''
vertices['position'] = [
    
                            #CATAVENTO
                            (+0.0, +0.0), # vertice 1 RETANGULO
                            (+0.3, +0.0), # vertice 2
                            (+0.3, +6.0), # vertice 3
                            (+0.0, +6.0), # vertice 4
                            (+0.0, +0.0), # vertice 5
    
                            (+0.15, +6.0), # vertice 6 TRIANGULOS
                            (+2.0, +7.0), # vertice 7
                            (+2.0, +5.0), # vertice 8
    
                            (+0.15, +6.0), # vertice 9
                            (+1.15, +8.0), # vertice 10
                            (-0.75, +8.0), # vertice 11
    
                            (+0.15, +6.0), # vertice 12
                            (-2.0, +7.0), # vertice 13
                            (-2.0, +5.0), # vertice 14
    
                            (+0.15, +6.0), # vertice 15    
                            (+1.15, +4.0), # vertice 16    
                            (-0.75, +4.0), # vertice 17
    
                            #CASINHA
                            (+5.0, +0.0), # vertice 18 PAREDE
                            (+9.0, +0.0), # vertice 19
                            (+9.0, +6.0), # vertice 20
                            (+5.0, +6.0), # vertice 21
                            (+5.0, +0.0), # vertice 22
    
                            (+5.9, +3.9), # vertice 23 JANELA - batente
                            (+7.1, +3.9), # vertice 24
                            (+7.1, +5.1), # vertice 25
                            (+5.9, +5.1), # vertice 26
                            (+5.9, +3.9), # vertice 27
    
                            (+6.0, +4.0), # vertice 28 JANELA - vidro
                            (+7.0, +4.0), # vertice 29
                            (+7.0, +5.0), # vertice 30
                            (+6.0, +5.0), # vertice 31
                            (+6.0, +4.0), # vertice 32
    
                            (+6.0, +0.0), # vertice 33 PORTA
                            (+8.0, +0.0), # vertice 34
                            (+8.0, +3.5), # vertice 35
                            (+6.0, +3.5), # vertice 36
                            (+6.0, +0.0), # vertice 37
                            
                            (+4.5, +6.0), # vertice 38 TELHADO
                            (+9.5, +6.0), # vertice 39
                            (+7.0, +9.0), # vertice 40
                        ]

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [13]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [14]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [15]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [16]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [17]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

###  Novidade aqui! Vamos pegar a localização da variável color (uniform) para que possamos alterá-la em nosso laço da janela!

In [18]:
loc_color = glGetUniformLocation(program, "color")
R = 0.0
G = 0.0
B = 0.0

### Capturando eventos de teclado e modificando variáveis para a matriz de transformação

In [19]:
# exemplo para matriz de translacao

# translação
t_x = 0
t_y = 0

# angulo de rotação
r_ang = 0

# coeficiente de escala
e_x = 0
e_y = 0

def key_event(window,key,scancode,action,mods):
    global t_x, t_y, e_x, e_y
    
#     print('[key event] key=',key)
#     print('[key event] scancode=',scancode)
#     print('[key event] action=',action)
#     print('[key event] mods=',mods)
#     print('-------')

    if key == 262: t_x += 0.01 #seta direita
    if key == 263: t_x -= 0.01 #seta esquerda
        
    if key == 265: t_y += 0.01 #seta cima
    if key == 264: t_y -= 0.01 #seta baixo
        
    if (key == 32): #espaço
        e_x += 0.01 
        e_y += 0.01
        
    if (key == 342): #alt esquerdo
        e_x -= 0.01 
        e_y -= 0.01 
    
glfw.set_key_callback(window,key_event)

def mouse_button_callback(window,button,action,mods):
    global r_ang
    
    if (button == 1): #botão direito do mouse
        r_ang -= 0.1 
        
    if (button == 0): #botão esquerdo do mouse
        r_ang += 0.1 
    
glfw.set_mouse_button_callback(window,mouse_button_callback)

### Nesse momento, nós exibimos a janela!


In [20]:
glfw.show_window(window)

#### Função que transforma matriz de rotação ou de escala com ponto de referência (transladando antes e depois da transformação) 

In [21]:
def transf_ponto_referencia(matriz,ref_x,ref_y):

    matriz_trans = np.matrix([         [1.0, 0.0, 0.0, ref_x], 
                                       [0.0, 1.0, 0.0, ref_y], 
                                       [0.0, 0.0, 1.0, 0.0], 
                                       [0.0, 0.0, 0.0, 1.0]], np.float32)
    
    matriz_trans_back = np.matrix([    [1.0, 0.0, 0.0, -1*ref_x], 
                                       [0.0, 1.0, 0.0, -1*ref_y], 
                                       [0.0, 0.0, 1.0, 0.0], 
                                       [0.0, 0.0, 0.0, 1.0]], np.float32)
    
    return matriz_trans @ matriz @ matriz_trans_back

#### Função adicional que multiplica duas matrizes:

In [22]:
def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dor(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

A novidade agora é a função glDrawArrays()

Tal função recebe o tipo de primitiva (GL_TRIANGLES), o índice inicial do array de vértices (vamos mostrar todos os três vértices, por isso começamos com 0) e a quantidade de vértices ( len(vertices) ).

### Função que multiplica as 3 matrizes de transformação (translação, escada e rotação)

In [23]:
def mat_transf_total(rotate, translation_R, translation_L, scale_R, scale_L):
    #Translação pelos botões de seta
    mat_translation = np.matrix([   [1.0, 0.0, 0.0, translation_R], 
                                    [0.0, 1.0, 0.0, translation_L], 
                                    [0.0, 0.0, 1.0,     0.0      ], 
                                    [0.0, 0.0, 0.0,     1.0     ]], np.float32)
    
    #Escala pelos botões shift e space
    mat_scale       = np.matrix([   [scale_R,   0.0  , 0.0, 0.0], 
                                    [0.0,     scale_L, 0.0, 0.0], 
                                    [0.0,       0.0  , 1.0, 0.0], 
                                    [0.0,       0.0  , 0.0, 1.0]], np.float32)
    
    #Rotação pelos botões direito e esquerdo do mouse
    angulo = 10*rotate
    rad = math.radians(angulo) 
    c = math.cos(rad)
    s = math.sin(rad)
    mat_rotate     = np.matrix([ [ c , -s , 0.0, 0.0], 
                                 [ s ,  c , 0.0, 0.0], 
                                 [0.0, 0.0, 1.0, 0.0], 
                                 [0.0, 0.0, 0.0, 1.0]], np.float32)
    
    
    return mat_translation @ mat_scale @ mat_rotate

In [24]:
r_ang = 0.01

t_x = 0
t_y = 0

e_x = 1
e_y = 1

while not glfw.window_should_close(window):
    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
  
    #Draw Triangle
    #Translação pelos botões de seta
    mat_translation = np.matrix([   [1.0, 0.0, 0.0, t_x], 
                                    [0.0, 1.0, 0.0, t_y], 
                                    [0.0, 0.0, 1.0, 0.0], 
                                    [0.0, 0.0, 0.0, 1.0]], np.float32)
    
    #Escala pelos botões shift e space
    s_x = e_x
    s_y = e_y
    mat_scale       = np.matrix([   [s_x, 0.0, 0.0, 0.0], 
                                    [0.0, s_y, 0.0, 0.0], 
                                    [0.0, 0.0, 1.0, 0.0], 
                                    [0.0, 0.0, 0.0, 1.0]], np.float32)
    
    #Rotação pelos botões direito e esquerdo do mouse
    angulo = 10*r_ang
    rad = math.radians(angulo) 
    c = math.cos(rad)
    s = math.sin(rad)
    mat_rotate     = np.matrix([   [c,  -s, 0.0, 0.0], 
                                   [s,   c, 0.0, 0.0], 
                                 [0.0, 0.0, 1.0, 0.0], 
                                 [0.0, 0.0, 0.0, 1.0]], np.float32)
    
    
    mat_identidade = np.matrix([    [1.0, 0.0, 0.0, 0.0], 
                                    [0.0, 1.0, 0.0, 0.0], 
                                    [0.0, 0.0, 1.0, 0.0], 
                                    [0.0, 0.0, 0.0, 1.0]], np.float32)
    
    #matriz que translada, escala e rotaciona
    matrizTransf = mat_transf_total(r_ang, t_x, t_y, e_x, e_y)
    
    
    #CATAVENTO
    glUniform4f(loc_color, R, G, B, 1.0) ### preto
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_identidade)
    glDrawArrays(GL_TRIANGLE_STRIP, 0, 5)
    

    glUniform4f(loc_color, 1, 0, 0, 1.0) ### vermelho
    #loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, transf_ponto_referencia(mat_rotate, 6, 0.15))
    glDrawArrays(GL_TRIANGLES, 5, 12)
 
    #CASINHA
    glUniform4f(loc_color, 1, 0.75, 0.75, 1.0) ### rosa
    #loc = glGetUniformLocation(program, "mat_transformation")
    #glUniformMatrix4fv(loc, 1, GL_TRUE, mat_identidade)
    glDrawArrays(GL_TRIANGLE_STRIP, 17, 5)
    
    glUniform4f(loc_color, R, G, B, 1.0) ### preta
    #loc = glGetUniformLocation(program, "mat_transformation")
    #glUniformMatrix4fv(loc, 1, GL_TRUE, mat_identidade)
    glDrawArrays(GL_TRIANGLE_STRIP, 22, 5)
    
    glUniform4f(loc_color, 1, 1, 0, 1.0) ### amarela (vai piscar entre amarelo e preto)
    #loc = glGetUniformLocation(program, "mat_transformation")
    #glUniformMatrix4fv(loc, 1, GL_TRUE, mat_identidade)
    glDrawArrays(GL_TRIANGLE_STRIP, 27, 5)
    
    glUniform4f(loc_color, 0.8, 0.5, 0, 1.0) ### marrom
    #loc = glGetUniformLocation(program, "mat_transformation")
    #glUniformMatrix4fv(loc, 1, GL_TRUE, mat_identidade)
    glDrawArrays(GL_TRIANGLE_STRIP, 32, 5)
    
    glUniform4f(loc_color, 1, 0, 0, 1.0) ### vermelho
    #loc = glGetUniformLocation(program, "mat_transformation")
    #glUniformMatrix4fv(loc, 1, GL_TRUE, mat_identidade)
    glDrawArrays(GL_TRIANGLES, 37, 3)

    glfw.swap_buffers(window)

glfw.terminate()